MultiRAT analysis code
================
Joanes Grandjean

![rat art](../assets/img/rat_art.png)

# Foreword

This is a jupyter notebook which contains all the code for reproducing my
analysis. The code is meant to be followed step-wise. 

If re-using some of the scripts, please follow citations guidelines for
the software used. I’ve provided the links to the software wherever
possible. See also the [license](../LICENSE.md) for this software.


This script was used to download and clean the Lui2020 awake rat dataset for comparison with the standardRat collection. bash script, because it is. Poorly documented, because it is for revisions. :wink: 

First we download the data and unzip it

In [ ]:
!cd $SCRATCH
!mkdir -p awake_rat/zip
!mkdir -p awake_rat/data
!mkdir -p awake_rat/bids
!cd awake_rat/zip

!wget -O part1.zip https://www.nitrc.org/frs/downloadlink.php/11762
!wget -O part2.zip https://www.nitrc.org/frs/downloadlink.php/11763
!wget -O part3.zip https://www.nitrc.org/frs/downloadlink.php/11764
!wget -O part4.zip https://www.nitrc.org/frs/downloadlink.php/11765
!wget -O part5.zip https://www.nitrc.org/frs/downloadlink.php/11766
!wget -O part6.zip https://www.nitrc.org/frs/downloadlink.php/11767

!ls .zip | while read line
!do
!unzip -o ../data $line
!done


Transform the awake rat dataset to BIDS using the information contained in the JSON files. Careful, because the data is not accurate (as of version 1.1). Some folders had to be manually deleted because of wrong information about the RARE anatomical files. 

In [ ]:
!cd ../data

!ls | while read rat
!do
!cd $rat 
!scan_screen=$(python3 -c 'import json,sys;JSON=json.load(sys.stdin);print(len(JSON["scans"])-1);' < ${rat}_info.json)
!anat=$(seq 0 $scan_screen | while read line; do python3 -c 'import json,sys;JSON=json.load(sys.stdin); protocol=JSON["scans"]['${line}']["protocol"];scan=JSON["scans"]['${line}']["name_in_raw"]; test="RARE" in protocol; print(scan) if test else print("")' < ${rat}_info.json; done)
!set $anat

!mkdir -p ../../bids/sub-${rat}/anat
!mkdir -p ../../bids/sub-${rat}/func

!cd raw/
!cp ${1} ../../../bids/sub-${rat}/anat/sub-${rat}_T2w.nii
!cd .. 


!ls rfmri_unprocessed/ | while read fmri
!do
!fmri_noext="${fmri%.*}"
!cp rfmri_unprocessed/${fmri} ../../bids/sub-${rat}/func/sub-${rat}_task-rest_run-${fmri_noext}_bold.nii
!done

!cd ..
!done

Preprocessing is then down with the job sumission code [here](./proj_submitjobs_awake.ipynb)   
Analysis is done [here](./proj_analysis_sba_awake.ipynb) 